In [ ]:
%matplotlib inline
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('/Users/jorge/Projects/finance')

from portfolio import Portfolio

In [ ]:
p = Portfolio()
days = p.performance.keys()
values = []
deposits = []
asset_returns = {}
for day, data in p.performance.items():
    values.append(data['marketValue'])
    deposits.append(data['totalDeposits'])
    for asset, asset_data in data['assets'].items():
        asset_returns.setdefault(asset, [])
        asset_returns[asset].append(asset_data['totalReturns'])
    print(f"{day}: {data['marketValue']:10,.2f}")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
plt.subplot(221)
plt.ylabel('Amounts')
plt.ylim(0, 30000)
plt.title('Market Value')
values_df = pd.DataFrame(values, index=days, columns=['Values'])
deposits_df = pd.DataFrame(deposits, index=days, columns=['Deposits'])
plt.plot(values_df, '#40C040', deposits_df, '#6060B0', linewidth=1.0)
ax = plt.gca()
yticks = ax.get_yticks()
ax.set_yticklabels(['{:10,.0f}'.format(y) for y in yticks])
ax.legend(['Market Value', 'Deposits'])

plt.subplot(222)
plt.title('True Time-Weighted Returns and Total Returns')
plt.ylabel('Returns')
ttwr = pd.DataFrame([x['ttwr'] for x in p.performance.values()], index=days, columns=['ttwr'])
returns = pd.DataFrame([(x['marketValue'] - x['totalDeposits']) / x['totalDeposits'] for x in p.performance.values()], index=days, columns=['returns'])
plt.plot(ttwr, '#40C040', returns, '#C08010', linewidth=1.0)
ax = plt.gca()
yticks = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(y * 100) for y in yticks])
ax.legend(['True Time-Weighted Returns', 'Total Returns'])
    
plt.subplot(223)
plt.title('Price Changes')
prices = pd.DataFrame(p.assets.prices).transpose()
changes = np.ones_like(prices) * (prices - prices.iloc[0]) / prices.iloc[0]
plt.plot(changes, linewidth=1.0)
ax = plt.gca()
yticks = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(y * 100) for y in yticks])
ax.legend(changes.columns)

plt.subplot(224)
plt.title('Correlations')
df = pd.DataFrame(p.assets.correlations)
df = df.sort_index(axis=1)

mask = np.zeros_like(df, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
ax = plt.gca()

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df, mask=mask, cmap=cmap, vmax=.3, square=True,
            linewidths=.5, cbar_kws={"shrink": .5}, annot=True, ax=ax)

plt.show()